In [ ]:
!pip install -r requirements.txt

# Parte 1 – Introducción al Tracking

In [ ]:
import mlflow
import statistics
import random

In [ ]:
mlflow.set_experiment("mlflow_basico")

def modelo_promedio(valores):
    return statistics.mean(valores)

with mlflow.start_run(run_name="ejemplo_promedio"):

    lista_valores = [random.randint(1, 100) for _ in range(10)]
    mlflow.log_param("tamaño_lista", len(lista_valores))
    mlflow.log_param("valores", lista_valores)

    resultado = modelo_promedio(lista_valores)
    mlflow.log_metric("promedio", resultado)

    with open("resultado.txt", "w") as f:
        f.write(f"Valores: {lista_valores}\n")
        f.write(f"Promedio: {resultado}\n")

    mlflow.log_artifact("resultado.txt")

# Parte 2 – Registro de Modelos de Machine Learning

In [ ]:
import mlflow
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score, f1_score, precision_score
import logging
from mlflow.tracking import MlflowClient

In [ ]:
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

mlflow.set_experiment("breast_cancer_experiment")

with mlflow.start_run(run_name = "breast_cancer_logistic_regression_v1"):

    X, y = load_breast_cancer(return_X_y= True)
    params = {"C": 0.1, "random_state": 42, "max_iter": 1000, "solver": "liblinear", "penalty": "l2"}
    mlflow.log_params(params)
    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))
    mlflow.log_metric("f1_score", f1_score(y, y_pred))
    mlflow.log_mectric("precision", precision_score(y, y_pred))
    mlflow.sklearn.log_model(lr, "breast_cancer_logistic_regression_model")
    result = mlflow.register_model(
        model_uri=f"runs:/{run.info.run_id}/breast_cancer_logistic_regression_model",
        name="bc_logistic_regression_model"
    )

In [ ]:
# Promover a Staging

client = MlflowClient()
model_name = "bc_logistic_regression_model"
version = 1

client.transition_model_version_stage(
    name=model_name,
    version=version,
    stage="Staging"
)

In [ ]:
# Promover a Production

client.transition_model_version_stage(
    name=model_name,
    version=version,
    stage="Production"
)

# Parte 3 – Tracking de Modelos de Lenguaje (LLMs)

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()
gemini_key = os.environ.get("GEMINI_API_KEY")

gemini = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/", api_key=gemini_key)
model_name_g = "gemini-2.5-flash-lite"
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
model_name_o = "gpt-oss:20b"


In [ ]:
import os
import time
import mlflow
from openai import OpenAI
from dotenv import load_dotenv
from mlflow.tracking import MlflowClient

load_dotenv()
gemini_key = os.environ.get("GEMINI_API_KEY")

gemini = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/", api_key=gemini_key)
ollama = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

# Prompt de prueba
prompt = "¿Que es la IA?"

# Simulación de costo por token
def estimar_costo(tokens):
    return tokens * 0.0001

def ejecutar_llm(nombre_modelo, cliente, proveedor, temperatura, tipo_tarea, registry_name):
    mlflow.set_experiment("comparacion_llm_chat")

    with mlflow.start_run(run_name=f"{registry_name}_run") as run:
        inicio = time.time()
        response = cliente.chat.completions.create(
            model=nombre_modelo,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperatura
        )
        fin = time.time()

        texto = response.choices[0].message.content
        tokens = response.usage.total_tokens
        latencia = fin - inicio
        costo = estimar_costo(tokens)

        # Log de parámetros
        mlflow.log_param("modelo", nombre_modelo)
        mlflow.log_param("temperatura", temperatura)
        mlflow.log_param("proveedor", proveedor)
        mlflow.log_param("tipo_tarea", tipo_tarea)

        # Log de métricas
        mlflow.log_metric("latencia", latencia)
        mlflow.log_metric("tokens", tokens)
        mlflow.log_metric("costo_estimado", costo)

        # Artefactos
        with open("prompt.txt", "w") as f:
            f.write(prompt)
        with open("respuesta.txt", "w") as f:
            f.write(texto)

        mlflow.log_artifact("prompt.txt")
        mlflow.log_artifact("respuesta.txt")

        # Registrar modelo en el Model Registry
        mlflow.register_model(
            model_uri=f"runs:/{run.info.run_id}/respuesta.txt",
            name=registry_name
        )

# Ejecutar ambos modelos
ejecutar_llm("gemini-2.5-flash-lite", gemini, "Gemini", 0.7, "chat", "llm_gemini_chat")
ejecutar_llm("gpt-oss:20b", ollama, "Ollama", 0.7, "chat", "llm_ollama_chat")


In [ ]:
# Promocion de los modelos a Staging

client = MlflowClient()

client.transition_model_version_stage(
    name="llm_gemini_chat",
    version=1,
    stage="Staging"
)

client.transition_model_version_stage(
    name="llm_ollama_chat",
    version=1,
    stage="Staging"
)
